# Report on weather data

In [ ]:
import sys
import os
import pandas as pd

In [13]:
# git directories

# Josh
gitdir = "/Users/joshd/Documents/GitHub/GA2024_1/"

#Magda
#gitdir =

# Add the directory
module_path = os.path.abspath(gitdir)
sys.path.append(module_path)

from home_messages_db import HomeMessagesDB

db = HomeMessagesDB('sqlite:///myhome.db')

df_st = db.get_smartthings()
df_p1e = db.get_p1e()
df_p1g = db.get_p1g()
df_weather = db.get_weather_data()




In [9]:
# get overview

# gas data
print(df_p1g.shape)

# energy usage (smartthings)
print(df_st.shape)

# weather
print(df_weather.shape)

(80716, 3)
(1182210, 9)
(20424, 9)


In [19]:
df_ins_temp = df_st[df_st["capability"] == "temperatureMeasurement"]

In [25]:
# data types
df_ins_temp['value'] = df_ins_temp['value'].astype(float)

# rename temperature
df_ins_temp.rename(columns={'value': 'temperature'}, inplace=True)

/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/2229040446.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp['value'] = df_ins_temp['value'].astype(float)
/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/2229040446.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp.rename(columns={'value': 'temperature'}, inplace=True)


In [27]:
# Convert epoch_seconds to datetime
df_ins_temp['datetime'] = pd.to_datetime(df_ins_temp['epoch'], unit='s')

# Select only the target column (temperature) and the epoch time
df_target = df_ins_temp[['datetime', 'temperature']]

# Set datetime as index
df_target.set_index('datetime', inplace=True)

# Resample by day and calculate the average temperature
daily_avg_df = df_target.resample('D').mean()

# Reset index to make 'datetime' a column again
daily_avg_df.reset_index(inplace=True)

/var/folders/z1/762t94_j4tx1bhtk_g1t9qh80000gn/T/ipykernel_21265/3282028656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ins_temp['datetime'] = pd.to_datetime(df_ins_temp['epoch'], unit='s')
